In [1]:
import os
import re
import numpy as np 
import shutil
from sklearn.metrics import accuracy_score

import transformers
from transformers import BertTokenizer, BertModel

import torch
from torch import cuda
from  tqdm.notebook import tqdm
device = 'cuda' if cuda.is_available() else 'cpu'
import pickle
import re
# from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk

In [47]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [38]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
#     text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [39]:
with open('cozmo_filtered_data.pkl', "rb") as fh:
    cozmo_data = pickle.load(fh)
    cozmo_data.rename(columns={'Interst/Alarm':'Interest/Alarm', 'Frusteration/Relief':'Frustration/Relief'}, inplace=True)
with open('misty_filtered_data.pkl', "rb") as fh:
    misty_data = pickle.load(fh)
    misty_data.rename(columns={'Interst/Alarm':'Interest/Alarm', 'Frusteration/Relief':'Frustration/Relief'}, inplace=True)

In [40]:
cozmo_data['Description'] = cozmo_data['Description'].apply(clean_text)

In [41]:
misty_data['Description'] = misty_data['Description'].apply(clean_text)

In [42]:
cols = ['Interest/Alarm','Confusion/Understanding','Frustration/Relief','Sorrow/Joy','Anger/Gratitude','Fear/Hope','Boredom/Surprise','Disgust/Desire']

In [43]:
targs = [col.split('/') for col in misty_data.columns if '/' in col]

targs = [i for x in targs for i in x]
targs = list(map(str.lower, targs))
targs

['interest',
 'alarm',
 'confusion',
 'understanding',
 'frustration',
 'relief',
 'sorrow',
 'joy',
 'anger',
 'gratitude',
 'fear',
 'hope',
 'boredom',
 'surprise',
 'disgust',
 'desire']

In [67]:
targs = ['interest',
 'alarm',
 'confusion',
 'understanding',
 'frustration',
 'relief',
 'sorrow',
 'joy',
 'anger',
 'gratitude',
 'fear',
 'hope',
 'boredom',
 'surprise',
 'disgust',
 'desire']

In [68]:
def rate(row, guess):
    points = 0.0
    if row[0] == 1:
        if guess == 'interest':
            points += 1
    if row[0] == 5:
        if guess == 'alarm':
            points += 1
    if row[0] == 2:
        if guess == 'interest':
            points += 0.5
    if row[0] == 4:
        if guess == 'alarm':
            points += 0.5    
            
    if row[1] == 1:
        if guess == 'confusion':
            points += 1
    if row[1] == 5:
        if guess == 'understanding':
            points += 1
    if row[1] == 2:
        if guess == 'confusion':
            points += 0.5
    if row[1] == 4:
        if guess == 'understanding':
            points += 0.5   
            
    if row[2] == 1:
        if guess == 'frustration':
            points += 1
    if row[2] == 5:
        if guess == 'relief':
            points += 1 
    if row[2] == 2:
        if guess == 'frustration':
            points += 0.5
    if row[2] == 4:
        if guess == 'relief':
            points += 0.5            
            
    if row[3] == 1:
        if guess == 'sorrow':
            points += 1
    if row[3] == 5:
        if guess == 'joy':
            points += 1   
    if row[3] == 2:
        if guess == 'sorrow':
            points += 0.5
    if row[3] == 4:
        if guess == 'joy':
            points += 0.5            
            
    if row[4] == 1:
        if guess == 'anger':
            points += 1
    if row[4] == 5:
        if guess == 'gratitude':
            points += 1   
    if row[4] == 2:
        if guess == 'anger':
            points += 0.5
    if row[4] == 4:
        if guess == 'gratitude':
            points += 0.5            
            
    if row[5] == 1:
        if guess == 'fear':
            points += 1
    if row[5] == 5:
        if guess == 'hope':
            points += 1   
    if row[5] == 2:
        if guess == 'fear':
            points += 0.5
    if row[5] == 4:
        if guess == 'hope':
            points += 0.5           
            
    if row[6] == 1:
        if guess == 'boredom':
            points += 1
    if row[6] == 5:
        if guess == 'surprise':
            points += 1 
    if row[6] == 2:
        if guess == 'boredom':
            points += 0.5
    if row[6] == 4:
        if guess == 'surprise':
            points += 0.5
            
    if row[7] == 1:
        if guess == 'disgust':
            points += 1
    if row[7] == 5:
        if guess == 'desire':
            points += 1   
    if row[7] == 2:
        if guess == 'disgust':
            points += 0.5
    if row[7] == 4:
        if guess == 'desire':
            points += 0.5
            
    return points
            
rate([1, 1, 3, 3, 3, 4, 4, 5], 'interest')       

1.0

In [71]:
correct = 0.0
total = 0.0

for i,row in tqdm(misty_data.iterrows()):
    sequence_to_classify = row.Description
    guess = classifier(sequence_to_classify, targs)
    guess = guess['labels'][0] 
    c = row[cols].values
    points = rate(c, guess)
    if points == 0.0: continue
    correct += points
    total += 1.0


0it [00:00, ?it/s]

cozmo (547)

In [70]:
correct/total, correct, total 

(0.6953781512605042, 165.5, 238.0)

misty results (547)

In [73]:
correct/total, correct, total 

(0.7927927927927928, 264.0, 333.0)